In [2]:
import pandas as pd
import numpy as np
import os, re, datetime


In [47]:
df = pd.read_csv("outOfJob.csv")

In [48]:
origin = '/pylon5/sy5fp1p/ehanna/logs/hosts/'

In [49]:
pattern = re.compile('(\d+-\d+-\d+)T(\d+:\d+:\d+)-(\d+:\d+)\s(.*?)\.pvt.*(-\s)+(.*)')

In [50]:
df = df.set_index('Unnamed: 0')
df.index.name = 'Index'

#### Out of Job Error analysis was important as the node was crashing when it was not even running a job on it. The reason for this being a cascading effect from other nodes which are all connected through same connection to the server

In [55]:
def outOfJobErrorText(df):
    corpus = []
    count = 0
    for i in range(len(df)):
        print(i)
        for column in df.columns:
            if df.loc[df.index[i],column]>0:
                date = df.index[i].split('_')[0]
                timeString = df.index[i].split('_')[1]
                time = datetime.datetime.strptime(timeString,'%H:%M:%S')
                timeToStartSeconds = (int(time.hour)*3600+int(time.minute)*60+int(time.second))-(1*60)
    #             print(timeToStartSeconds)
                timeToStart = str(datetime.timedelta(seconds=timeToStartSeconds))
                fileName = origin +column+'.pvt.bridges.psc.edu/'+date+'-'+column+'.pvt.bridges.psc.edu.log'
                with open(fileName) as file:
                    fileCorpus = ''
                    for line in file:
                        if 'not responding' not in line and 'nfs: server' not in line:
                            match = re.search(pattern,line)
                            if match is not None:
                                time = match.group(2)
                                if time>timeToStart:
                                    text = match.group(6)
                                    fileCorpus+=text+'\n'
                                elif time>timeString:
                                    break
                    corpus.append(fileCorpus)
    return corpus

In [1]:
corpus = outOfJobErrorText(df)

In [59]:
allText = ''
for i in corpus:
    if len(i)>0:
        allText+=i+'\n'
file = open("OutOfJobPredecessors.txt","w")
file.write(allText)
file.close()

In [4]:
textUniquePattern = re.compile('[a-zA-Z0-9\.\:]*[\d]+[a-zA-Z0-9\.\:]*')
testPattern = re.compile('\d+.\d+.\d+.\d+')

In [5]:
uniqueTexts = []
textFrequencies = []
with open("OutOfJobPredecessors.txt") as file:
    for line in file:
        line = line.strip('\n')
        line = re.sub(textUniquePattern,"",line)
        if line not in uniqueTexts:
            uniqueTexts.append(line)
            textFrequencies.append(1)
        else:
            textFrequencies[uniqueTexts.index(line)]+=1

In [10]:
allTexts = []
for i in range(len(uniqueTexts)):
    allTexts.append([uniqueTexts[i],textFrequencies[i]])

In [15]:
allTexts = sorted(allTexts,key=lambda x: x[1],reverse=True)

In [17]:
text = ''
for i in range(len(allTexts)):
    text+= allTexts[i][0]+' '+str(allTexts[i][1])+'\n'
file = open("OutOfJobuniquePred.txt","w")
file.write(text)
file.close()